In [153]:
import pandas as pd
import numpy
import pylab
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import LinearRegression

In [3]:
trainBodies = pd.read_csv('fncData/train_bodies.csv')
trainStances = pd.read_csv('fncData/train_stances.csv')
trainStancesRandom = pd.read_csv('fncData/train_stances.random.csv')

print "Train Bodies (20/%s):" % (len(trainBodies))
# trainBodies.head(20)

# to get articleBody:
# trainBodies.get_value(0, 'articleBody')

Train Bodies (20/1683):


In [4]:
print "Train Stances (20/%s):" % (len(trainStances))
# trainStances.head(20)

# trainStances.addColumn(stance === unrelated)

Train Stances (20/49972):


In [5]:
print "Train Stances Random (20/%s)" % (len(trainStancesRandom))
# trainStancesRandom.head(20)

Train Stances Random (20/49972)


In [176]:
# build feature vocabulary
ngram = 1
vectorizer = CountVectorizer(ngram_range=(1, ngram))

testStances = trainStances[0:20]

for index, row in testStances.iterrows():
    headline = row['Headline']
    bodyId = row['Body ID']
    stance = row['Stance']
    body = trainBodies[trainBodies['Body ID'] == bodyId]['articleBody'].item()
    
#     print index
#     print "bodyId: %s\n" % bodyId
#     print "HEADLINE: \n%s\n" % headline
#     print "BODY: \n%s\n" % body
#     print "STANCE: \n%s\n" % stance

    headline_words = headline.split()
    body_words = body.split()
    
    num_words_in_common = len(set(headline_words).intersection(body_words))
    
    results.append((num_words_in_common, stance))

print results 

# I'd like to graph the num_words_in_common integer to the stance classification, to see what kind of correlation we're working with here.

[('unrelated', 2), ('agree', 10), ('unrelated', 4), ('unrelated', 2), ('disagree', 6), ('agree', 14), ('unrelated', 0), ('unrelated', 2), ('agree', 2), ('unrelated', 1), ('discuss', 0), ('agree', 7), ('unrelated', 2), ('unrelated', 2), ('discuss', 4), ('unrelated', 7), ('discuss', 3), ('agree', 4), ('unrelated', 3), ('discuss', 2), (2, 'unrelated'), (10, 'agree'), (4, 'unrelated'), (2, 'unrelated'), (6, 'disagree'), (14, 'agree'), (0, 'unrelated'), (2, 'unrelated'), (2, 'agree'), (1, 'unrelated'), (0, 'discuss'), (7, 'agree'), (2, 'unrelated'), (2, 'unrelated'), (4, 'discuss'), (7, 'unrelated'), (3, 'discuss'), (4, 'agree'), (3, 'unrelated'), (2, 'discuss'), (2, 'unrelated'), (10, 'agree'), (4, 'unrelated'), (2, 'unrelated'), (6, 'disagree'), (14, 'agree'), (0, 'unrelated'), (2, 'unrelated'), (2, 'agree'), (1, 'unrelated'), (0, 'discuss'), (7, 'agree'), (2, 'unrelated'), (2, 'unrelated'), (4, 'discuss'), (7, 'unrelated'), (3, 'discuss'), (4, 'agree'), (3, 'unrelated'), (2, 'discuss'), 